We take as a whitelist the cell barcodes accepted in the first mapping.

In [2]:
star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    star_reference_dir='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna';
    sjdbgtf='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
        --runThreadN 24 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist '/lustre1/project/stg_00002/lcb/fderop/data/20220125_hydrop_data_analysis/HyDrop-RNA_publicdata_comparison/dropseq_whitelist/drop-seq_macosko_whitelist.txt' \
        --soloType CB_UMI_Simple \
        --soloCBmatchWLtype 1MM \
        --soloCBstart  1 \
        --soloCBlen 12 \
        --soloUMIstart 13 \
        --soloUMIlen 8\
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [ ]:
dir=demultiplexed_dropseq_sub_whitelist
mkdir $dir
for sample in fastq_dropseq_sub/*_2.SUB.fastq.gz
do
    NAME=${sample%_2.SUB.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq \
        ${NAME}_2.SUB.fastq.gz \
        ${NAME}_1.SUB.fastq.gz \
        $dir/$OUT.bam
done

fastq_dropseq_sub/SRR1853178
Feb 01 13:05:16 ..... started STAR run
Feb 01 13:05:16 ..... loading genome
Feb 01 13:05:50 ..... processing annotations GTF
Feb 01 13:05:56 ..... started mapping
Feb 01 13:33:33 ..... finished mapping
Feb 01 13:33:37 ..... started Solo counting
Feb 01 13:38:09 ..... finished Solo counting
Feb 01 13:38:09 ..... started sorting BAM
Feb 01 13:44:36 ..... finished successfully
fastq_dropseq_sub/SRR1853179
Feb 01 13:44:53 ..... started STAR run
Feb 01 13:44:54 ..... loading genome
Feb 01 13:45:26 ..... processing annotations GTF
Feb 01 13:45:31 ..... started mapping
Feb 01 14:24:20 ..... finished mapping
Feb 01 14:24:24 ..... started Solo counting
Feb 01 14:32:55 ..... finished Solo counting
Feb 01 14:32:55 ..... started sorting BAM


In [2]:
dir=demultiplexed_dropseq_sub_whitelist
mkdir $dir
for sample in fastq_dropseq_sub/*SRR1853182_2.SUB.fastq.gz
do
    NAME=${sample%_2.SUB.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_hydrop_rna_mapping_unique_cellfilter_dropseq \
        ${NAME}_2.SUB.fastq.gz \
        ${NAME}_1.SUB.fastq.gz \
        $dir/$OUT.bam
done

mkdir: cannot create directory ‘demultiplexed_dropseq_sub_whitelist’: File exists
fastq_dropseq_sub/SRR1853182
Feb 01 16:54:40 ..... started STAR run
Feb 01 16:54:42 ..... loading genome
Feb 01 16:55:34 ..... processing annotations GTF
Feb 01 16:55:40 ..... started mapping
Feb 01 17:50:00 ..... finished mapping
Feb 01 17:50:10 ..... started Solo counting
Feb 01 18:03:35 ..... finished Solo counting
Feb 01 18:03:35 ..... started sorting BAM
Feb 01 18:12:03 ..... finished successfully


Now, we can get a summary of the statistics below:

In [3]:
for sample in demultiplexed_dropseq/*/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

SRR1853178.Solo.out/Gene/Summary.csv
Number of Reads,510210716
Reads With Valid Barcodes,1
Sequencing Saturation,0.757257
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.62094
Reads Mapped to Genome: Unique+Multiple,0.545704
Reads Mapped to Genome: Unique,0.545704
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.372239
Reads Mapped to Transcriptome: Unique Genes,0.362322
Estimated Number of Cells,2629
Reads in Cells Mapped to Unique Genes,54743780
Fraction of Reads in Cells,0.296135
Mean Reads per Cell,20823
Median Reads per Cell,15973
UMIs in Cells,7773057
Mean UMI per Cell,2956
Median UMI per Cell,2202
Mean Genes per Cell,1651
Median Genes per Cell,1391
Total Genes Detected,22048

SRR1853179.Solo.out/Gene/Summary.csv
Number of Reads,524537760
Reads With Valid Barcodes,1
Sequencing Saturation,0.768773
Q30 Bases in CB+UMI,-nan
Q30 Bases in RNA read,0.620952
Reads Mapped to Genome: Unique+Multiple,0.543425
Reads Mapped to Genome: Unique,0.543425
Reads Mapped to Transcriptome: Uniq